In [1]:
import os
import os.path as path
import pandas as pd
import requests
from bs4 import BeautifulSoup
import regex as re
import time

In [2]:
def baseArchive():
    basePath = os.path.join(os.getcwd(), 'db', 'dataQSL.csv')
    os.makedirs(os.path.dirname(basePath), exist_ok=True)
    if not os.path.exists(basePath):
        with open(basePath, 'w', encoding='utf-8') as baseArchive:
            baseArchive.write('codigo,titulo,semestre,codigo_disciplina,nome_disciplina,professor,vagas_disciplina,vagas_disponiveis\n')  # Cabeçalho compatível com os dados
    return basePath



In [3]:
def readArchive():
    with open(baseArchive(), 'r', encoding='utf-8') as archive:
        data = archive.readlines()
    return data

In [ ]:
def get_course_info(esp_url):
    response = requests.get(esp_url)
    if response.status_code != 200:
        print(f"Erro ao acessar curso em {esp_url}: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    cadeiras_info = []

    content_div = soup.find('div', class_='abas-conteudo')
    turmas_section = content_div.find('div', id='turmas')
    versoes_section = turmas_section.find('div', class_="versao atual")

    content = versoes_section.find('div', class_='accordion cor-borda')
    
    content_section = content.find('table', class_="tabela-dados cor-borda")
    if not content_section:
        print("Erro: Tabela 'tabela-dados cor-borda' não encontrada.")
        return []

    for semestre in versoes_section.find_all('h3'):
        for linha in content_section.find_all('tr'):
            colunas = linha.find_all('td')
            if colunas:
                # Encontre todos os links na primeira coluna
                disciplina_link = colunas[0].find_all('a')
                semestre_nome = semestre.text.strip()
                
                # Verifique se há links na primeira coluna
                if disciplina_link:
                    for link in disciplina_link:
                        href = link.get('href', '')  # Obtém o atributo href do link
                        print(f"Verificando link: {href}")  # Adiciona depuração para verificar o valor de href
                        if href.startswith('/disciplinas/cod/'):  # Verifica se começa com '/disciplinas/cod/'
                            disciplina_text = link.get_text(strip=True)
                            disciplina_partes = disciplina_text.split(' - ', maxsplit=1)
                            codigo_disciplina = disciplina_partes[0].strip() if len(disciplina_partes) > 0 else None
                            nome_disciplina = disciplina_partes[1].strip() if len(disciplina_partes) > 1 else None
                        else:
                            print(f"Link ignorado, não começa com '/disciplinas/cod/': {href}")

                professor_span = colunas[0].find_all('span', class_='tabela-detalhe-info')
                professor_text = ''.join([span.text.strip() for span in professor_span])

                if professor_text:
                    professor = re.sub(r'(Professor responsável pela turma: |Professor Regente: )', '', professor_text).strip()
                    print(professor)
                else:
                    professor = None
                
                vagas_disc = colunas[-2].text.strip()
                vagas_disp = colunas[-1].text.strip()
                
                if vagas_disc.isdigit():
                    vagas_disciplina = colunas[-2].text.strip()
                if vagas_disp.isdigit():
                    vagas_disponiveis = colunas[-1].text.strip()
                
                # Adiciona os dados à lista
                cadeiras_info.append({
                    'semestre': semestre_nome,
                    'codigo_disciplina': codigo_disciplina,
                    'nome_disciplina': nome_disciplina,
                    'professor': professor,
                    'vagas_disciplina': vagas_disciplina,
                    'vagas_disponiveis': vagas_disponiveis
                })
                time.sleep(0.5)

    return cadeiras_info


Cursos de Graduação -> 
        Turmas Ofertadas -> 
                Nome das Disciplinas, 
                Nome dos Professores,  
                Número de Vagas Ofertadas e 
                Número de Alunos Matriculados




In [ ]:
def get_info(model_url):
    response = requests.get(model_url)
    if response.status_code != 200:
        print(f"Erro ao acessar {model_url}: {response.status_code}")
        return None 

    soup = BeautifulSoup(response.content, 'html.parser')
    cursos_data = []
    codigos_desejados = ['3900', '3910']

    content_div = soup.find('div', class_='abas-conteudo')
    graduacao_section = content_div.find('div', id="graduacao") if content_div else None
    cursos_list = graduacao_section.find_next('ul') if graduacao_section else None

    if cursos_list:
        cursos_links = cursos_list.find_all('a')
        for link in cursos_links:
            curso_codigo = link['href'].split('/')[-1]
            if curso_codigo:
                curso_titulo = link.text.strip()
                curso_url = f'https://institucional.ufpel.edu.br{link["href"]}'
                
                cadeiras = get_course_info(curso_url)
                for cadeira in cadeiras:
                    cursos_data.append({
                        'codigo': curso_codigo,
                        'titulo': curso_titulo,
                        **cadeira
                    })
    else:
        print("Nenhuma lista de cursos encontrada.")

    return cursos_data


In [ ]:
def saveData(data, filename=None):
    if filename is None:
        filename = baseArchive()

    df = pd.DataFrame(data)

    try:
        # Verifica se o arquivo já existe
        if os.path.exists(filename):
            # Lê o cabeçalho do arquivo existente para verificar a compatibilidade
            existing_df = pd.read_csv(filename, nrows=0)
            if list(existing_df.columns) != list(df.columns):
                raise ValueError("As colunas do novo DataFrame não correspondem ao arquivo existente.")
        
        # Salva os dados no arquivo (adiciona ao final se já existir)
        df.to_csv(filename, mode='a', header=not os.path.exists(filename), index=False)
        print(f"Dados salvos no arquivo '{filename}' com sucesso!")
    except Exception as e:
        print(f"Erro ao salvar dados: {e}\n colunas existentes {existing_df.columns}\ncolunas df {df.columns} ")


In [7]:
if __name__ == '__main__':
    # URL da página a ser capturada
    url_base = 'https://institucional.ufpel.edu.br/unidades/id/443'
    data = get_info(url_base)

    if data:
        saveData(data)

Verificando link: /disciplinas/cod/22000445
Verificando link: #
Link ignorado, não começa com '/disciplinas/cod/': #
MARIANA HÄRTER REMIÃO
Verificando link: /disciplinas/cod/22000442
Verificando link: #
Link ignorado, não começa com '/disciplinas/cod/': #
LUCIANA BICCA DODE
Verificando link: /disciplinas/cod/22000442
Verificando link: #
Link ignorado, não começa com '/disciplinas/cod/': #
LUCIANA BICCA DODE
Verificando link: /disciplinas/cod/22000447
Verificando link: #
Link ignorado, não começa com '/disciplinas/cod/': #
MARTA GONCALVES AMARALMARIANA HÄRTER REMIÃO
Verificando link: /disciplinas/cod/22000448
Verificando link: #
Link ignorado, não começa com '/disciplinas/cod/': #
LUCIELLI SAVEGNAGO
Verificando link: /disciplinas/cod/22000443
Verificando link: #
Link ignorado, não começa com '/disciplinas/cod/': #
SIBELE BORSUKFABIANA KOMMLING SEIXAS
Verificando link: /disciplinas/cod/22000444
Verificando link: #
Link ignorado, não começa com '/disciplinas/cod/': #
FABRICIO ROCHEDO CONC